In [1]:
# Configuración inicial y librerías
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime
from pathlib import Path
from scipy.optimize import minimize
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configuración
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:.4f}' if pd.notna(x) else '')
np.random.seed(42)

print("✅ Configuración completada")
print(f"📊 Sistema listo para análisis financiero")

✅ Configuración completada
📊 Sistema listo para análisis financiero


## 1. Carga y Validación de Datos

Carga de datos históricos desde Excel y complemento con APIs financieras.

In [2]:
# CARGA DE DATOS PRINCIPAL - VERSIÓN LIMPIA
def cargar_datos_reales():
    """Carga el dataset completo con todos los activos"""
    print("🔄 Cargando dataset completo...")
    
    # Usar datos cargados previamente
    if 'df_data_completo' in globals() and not df_data_completo.empty:
        return df_data_completo.copy(), activos_completos
    else:
        print("⚠️  Ejecutar celda anterior para cargar datos")
        return pd.DataFrame(), []

# Ejecutar carga
print("🚀 SISTEMA DE DATOS:")
df_data, activos = cargar_datos_reales()

if not df_data.empty:
    print(f"✅ Dataset activo con {len(activos)} activos:")
    print(f"   {', '.join(activos)}")
    print(f"   Período: {df_data['Fecha'].min().strftime('%d/%m/%Y')} → {df_data['Fecha'].max().strftime('%d/%m/%Y')}")
else:
    print("❌ Error: Dataset no disponible")

🚀 SISTEMA DE DATOS:
🔄 Cargando dataset completo...
⚠️  Ejecutar celda anterior para cargar datos
❌ Error: Dataset no disponible


In [3]:
# CARGA DE DATOS COMPLETA Y LIMPIA
def cargar_todos_los_activos():
    """Carga y combina datos del Excel con tickers adicionales de Yahoo Finance"""
    
    print("CARGANDO DATASET COMPLETO...")
    
    # 1. Datos base del Excel
    ARCHIVO_DATOS = r"c:\Users\trico\OneDrive\UBA\Management Financiero Bursatil\Cartera final\Datos_historicos_de_la_cartera.xlsx"
    df_excel = pd.read_excel(ARCHIVO_DATOS, sheet_name='Hoja1')
    df_excel['Fecha'] = pd.to_datetime(df_excel['Fecha'])
    
    # Filtrar solo columnas de precios
    activos_excel = [col for col in df_excel.columns 
                    if col != 'Fecha' and not col.startswith(('Retorno_', 'Tipo_', 'Tasa_'))]
    
    df_final = df_excel[['Fecha'] + activos_excel].copy()
    print(f"✅ Datos del Excel: {activos_excel}")
    
    # 2. Tickers adicionales de Yahoo Finance
    tickers_adicionales = {
        'BHIL': 'BHIP.BA',  # BHIL encontrado como BHIP.BA
        'METRO': 'METR.BA', # METRO encontrado como METR.BA  
        'IBM': 'IBM'        # IBM directo
    }
    
    # Obtener período de datos del Excel
    fecha_min = df_final['Fecha'].min()
    fecha_max = df_final['Fecha'].max()
    
    for nombre, ticker in tickers_adicionales.items():
        try:
            print(f"🔄 Descargando {nombre} ({ticker})...")
            data = yf.download(ticker, start=fecha_min, end=fecha_max + pd.Timedelta(days=1), progress=False)
            
            if len(data) > 0:
                # Preparar datos
                ticker_df = data['Close'].reset_index()
                ticker_df.columns = ['Fecha', nombre]
                ticker_df['Fecha'] = pd.to_datetime(ticker_df['Fecha'])
                
                # Integrar con datos principales
                df_final = pd.merge(df_final, ticker_df, on='Fecha', how='outer')
                print(f"✅ {nombre}: {len(data)} registros agregados")
            else:
                print(f"❌ {nombre}: Sin datos disponibles")
                
        except Exception as e:
            print(f"❌ Error con {nombre}: {str(e)[:50]}")
    
    # 3. Finalizar dataset
    df_final = df_final.sort_values('Fecha').reset_index(drop=True)
    activos_finales = [col for col in df_final.columns if col != 'Fecha']
    
    print(f"\n🎯 DATASET FINAL:")
    print(f"   • Total activos: {len(activos_finales)}")
    print(f"   • Activos: {activos_finales}")
    print(f"   • Registros: {len(df_final)}")
    print(f"   • Período: {df_final['Fecha'].min().strftime('%d/%m/%Y')} → {df_final['Fecha'].max().strftime('%d/%m/%Y')}")
    
    return df_final, activos_finales

# Ejecutar carga completa
df_data_completo, activos_completos = cargar_todos_los_activos()

# Verificar integridad de datos
print(f"\nVERIFICACIÓN DE DATOS:")
for activo in activos_completos:
    valid_count = df_data_completo[activo].notna().sum()
    print(f"   {activo:8s}: {valid_count:3d} registros válidos")

CARGANDO DATASET COMPLETO...
✅ Datos del Excel: ['AAPL', 'SPY', 'EWZ', 'CEPU']
🔄 Descargando BHIL (BHIP.BA)...
✅ Datos del Excel: ['AAPL', 'SPY', 'EWZ', 'CEPU']
🔄 Descargando BHIL (BHIP.BA)...
✅ BHIL: 161 registros agregados
🔄 Descargando METRO (METR.BA)...
✅ BHIL: 161 registros agregados
🔄 Descargando METRO (METR.BA)...
✅ METRO: 161 registros agregados
🔄 Descargando IBM (IBM)...
✅ METRO: 161 registros agregados
🔄 Descargando IBM (IBM)...
✅ IBM: 165 registros agregados

🎯 DATASET FINAL:
   • Total activos: 7
   • Activos: ['AAPL', 'SPY', 'EWZ', 'CEPU', 'BHIL', 'METRO', 'IBM']
   • Registros: 172
   • Período: 03/01/2025 → 02/09/2025

VERIFICACIÓN DE DATOS:
   AAPL    : 161 registros válidos
   SPY     : 161 registros válidos
   EWZ     : 161 registros válidos
   CEPU    : 161 registros válidos
   BHIL    : 161 registros válidos
   METRO   : 161 registros válidos
   IBM     : 165 registros válidos
✅ IBM: 165 registros agregados

🎯 DATASET FINAL:
   • Total activos: 7
   • Activos: ['AAP

## 2. Análisis de Riesgo y Rendimiento

Cálculo de métricas fundamentales de riesgo para cada activo.

In [4]:
class RiskCalculator:
    """Calculadora profesional de métricas de riesgo"""
    
    def __init__(self, risk_free_rate=0.02):
        self.rf_rate = risk_free_rate
    
    def calculate_metrics(self, returns):
        """Calcula métricas completas de riesgo"""
        if len(returns) < 10:
            return {}
        
        # Métricas básicas
        annual_return = returns.mean() * 252
        volatility = returns.std() * np.sqrt(252)
        
        # Ratios
        sharpe = (annual_return - self.rf_rate) / volatility if volatility > 0 else 0
        downside_returns = returns[returns < 0]
        sortino = ((annual_return - self.rf_rate) / (downside_returns.std() * np.sqrt(252))) if len(downside_returns) > 0 else 0
        
        # VaR y CVaR
        var_95 = np.percentile(returns, 5)
        cvar_95 = returns[returns <= var_95].mean()
        
        # Maximum Drawdown
        cumulative = (1 + returns).cumprod()
        running_max = cumulative.cummax()
        drawdown = (cumulative - running_max) / running_max
        max_dd = drawdown.min()
        
        return {
            'rendimiento_anual': annual_return,
            'volatilidad': volatility,
            'sharpe_ratio': sharpe,
            'sortino_ratio': sortino,
            'var_95': var_95,
            'cvar_95': cvar_95,
            'max_drawdown': max_dd,
            'skewness': returns.skew(),
            'kurtosis': returns.kurtosis()
        }

def analizar_activos(df, activos):
    """Analiza todos los activos y retorna métricas"""
    calculator = RiskCalculator()
    resultados = []
    
    for activo in activos:
        precios = df[activo].dropna()
        if len(precios) > 30:
            returns = precios.pct_change().dropna()
            metricas = calculator.calculate_metrics(returns)
            if metricas:
                metricas['activo'] = activo
                metricas['observaciones'] = len(returns)
                resultados.append(metricas)
    
    df_metricas = pd.DataFrame(resultados)
    
    # Reordenar columnas
    cols = ['activo', 'observaciones', 'rendimiento_anual', 'volatilidad', 
            'sharpe_ratio', 'sortino_ratio', 'max_drawdown', 'var_95', 'cvar_95']
    df_metricas = df_metricas[cols + [c for c in df_metricas.columns if c not in cols]]
    
    return df_metricas

# Calcular métricas
print("📊 Calculando métricas de riesgo...")
df_metricas = analizar_activos(df_data, activos)

# Mostrar resultados formateados
if not df_metricas.empty:
    df_display = df_metricas.copy()
    
    # Formatear porcentajes
    for col in ['rendimiento_anual', 'volatilidad', 'max_drawdown', 'var_95', 'cvar_95']:
        df_display[col] = (df_display[col] * 100).round(2)
    
    # Formatear ratios
    for col in ['sharpe_ratio', 'sortino_ratio']:
        df_display[col] = df_display[col].round(3)
    
    print(f"\n📈 Métricas de Riesgo (n={len(df_metricas)} activos):")
    print("─" * 80)
    
    # Usar display() para mejor visualización
    from IPython.display import display
    display(df_display[['activo', 'rendimiento_anual', 'volatilidad', 'sharpe_ratio', 'max_drawdown']])
    
    # Top performers
    mejor_sharpe = df_metricas.loc[df_metricas['sharpe_ratio'].idxmax()]
    print(f"\n🏆 Mejor Sharpe Ratio: {mejor_sharpe['activo']} ({mejor_sharpe['sharpe_ratio']:.3f})")
else:
    print("❌ No se pudieron calcular métricas")

📊 Calculando métricas de riesgo...


KeyError: "None of [Index(['activo', 'observaciones', 'rendimiento_anual', 'volatilidad',\n       'sharpe_ratio', 'sortino_ratio', 'max_drawdown', 'var_95', 'cvar_95'],\n      dtype='object')] are in the [columns]"

## 3. Visualizaciones y Análisis de Correlación

Gráficos profesionales para análisis de performance y correlaciones.

In [ ]:
def crear_visualizaciones(df, activos):
    """Crear visualizaciones profesionales"""
    
    # 1. Performance normalizada
    fig_perf = go.Figure()
    colors = px.colors.qualitative.Set3
    
    for i, activo in enumerate(activos):
        precios = df[activo].dropna()
        if len(precios) > 10:
            fechas = df.loc[df[activo].notna(), 'Fecha']
            perf_norm = (precios / precios.iloc[0]) * 100
            
            fig_perf.add_trace(go.Scatter(
                x=fechas, y=perf_norm, name=activo,
                line=dict(width=2, color=colors[i % len(colors)]),
                hovertemplate=f'{activo}: %{{y:.1f}}<extra></extra>'
            ))
    
    fig_perf.update_layout(
        title='Performance Normalizada de Activos (Base 100)',
        xaxis_title='Fecha', yaxis_title='Performance',
        template='plotly_white', height=500, hovermode='x unified'
    )
    # Mostrar y guardar como imagen estática
    fig_perf.show()
    fig_perf.write_image("performance_normalizada.png", width=1000, height=500, scale=2)
    
    # 2. Mapa Riesgo vs Rendimiento
    if not df_metricas.empty:
        fig_risk = go.Figure()
        
        for i, row in df_metricas.iterrows():
            fig_risk.add_trace(go.Scatter(
                x=[row['volatilidad'] * 100],
                y=[row['rendimiento_anual'] * 100],
                mode='markers+text',
                text=[row['activo']],
                textposition="top center",
                name=row['activo'],
                marker=dict(size=12, color=colors[i % len(colors)]),
                showlegend=False,
                hovertemplate=f"<b>{row['activo']}</b><br>Rendimiento: {row['rendimiento_anual']*100:.1f}%<br>Volatilidad: {row['volatilidad']*100:.1f}%<extra></extra>"
            ))
        
        fig_risk.update_layout(
            title='Mapa Riesgo vs Rendimiento',
            xaxis_title='Volatilidad Anual (%)',
            yaxis_title='Rendimiento Anual (%)',
            template='plotly_white', height=500
        )
        fig_risk.add_hline(y=0, line_dash="dash", line_color="gray")
        # Mostrar y guardar como imagen estática
        fig_risk.show()
        fig_risk.write_image("mapa_riesgo_rendimiento.png", width=1000, height=500, scale=2)
    
    # 3. Matriz de correlación
    returns_matrix = pd.DataFrame()
    for activo in activos:
        precios = df[activo].dropna()
        if len(precios) > 30:
            returns = precios.pct_change().dropna()
            returns_matrix[activo] = returns
    
    if len(returns_matrix.columns) > 1:
        corr_matrix = returns_matrix.corr()
        
        fig_corr = go.Figure(data=go.Heatmap(
            z=corr_matrix.values,
            x=corr_matrix.columns,
            y=corr_matrix.index,
            colorscale='RdBu', zmid=0,
            text=corr_matrix.round(2).values,
            texttemplate='%{text}',
            hovertemplate='%{y} vs %{x}: %{z:.3f}<extra></extra>'
        ))
        
        fig_corr.update_layout(
            title='Matriz de Correlación de Retornos',
            template='plotly_white', height=400
        )
        # Mostrar y guardar como imagen estática
        fig_corr.show()
        fig_corr.write_image("matriz_correlacion.png", width=800, height=400, scale=2)
        
        # Estadísticas de correlación
        corr_mean = corr_matrix.mean().mean()
        print(f"📊 Correlación promedio: {corr_mean:.3f}")
        
        return returns_matrix
    
    return pd.DataFrame()

# Crear visualizaciones
print("📊 Generando visualizaciones...")
returns_data = crear_visualizaciones(df_data, activos)

📊 Generando visualizaciones...


ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


## 4. Optimización de Cartera

Cálculo de carteras óptimas usando teoría moderna de portafolios.

In [ ]:
class PortfolioOptimizer:
    """Optimizador profesional de carteras"""
    
    def __init__(self, returns_data, risk_free_rate=0.02):
        self.returns = returns_data.dropna()
        self.assets = list(self.returns.columns)
        self.n_assets = len(self.assets)
        self.rf_rate = risk_free_rate / 252
        self.mean_returns = self.returns.mean()
        self.cov_matrix = self.returns.cov()
    
    def portfolio_performance(self, weights):
        """Calcula rendimiento y riesgo del portafolio"""
        ret = np.sum(self.mean_returns * weights) * 252
        vol = np.sqrt(np.dot(weights.T, np.dot(self.cov_matrix * 252, weights)))
        return ret, vol
    
    def negative_sharpe(self, weights):
        """Función objetivo para maximizar Sharpe"""
        ret, vol = self.portfolio_performance(weights)
        return -(ret - self.rf_rate * 252) / vol
    
    def optimize_sharpe(self):
        """Optimiza para máximo Sharpe Ratio"""
        constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
        bounds = tuple((0, 1) for _ in range(self.n_assets))
        initial = np.array([1/self.n_assets] * self.n_assets)
        
        result = minimize(self.negative_sharpe, initial, method='SLSQP', 
                         bounds=bounds, constraints=constraints)
        
        if result.success:
            weights = result.x
            ret, vol = self.portfolio_performance(weights)
            sharpe = (ret - self.rf_rate * 252) / vol
            
            return {
                'tipo': 'Máximo Sharpe',
                'weights': dict(zip(self.assets, weights)),
                'rendimiento': ret,
                'volatilidad': vol,
                'sharpe_ratio': sharpe
            }
        return None
    
    def optimize_min_vol(self):
        """Optimiza para mínima volatilidad"""
        def portfolio_vol(weights):
            return self.portfolio_performance(weights)[1]
        
        constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
        bounds = tuple((0, 1) for _ in range(self.n_assets))
        initial = np.array([1/self.n_assets] * self.n_assets)
        
        result = minimize(portfolio_vol, initial, method='SLSQP',
                         bounds=bounds, constraints=constraints)
        
        if result.success:
            weights = result.x
            ret, vol = self.portfolio_performance(weights)
            sharpe = (ret - self.rf_rate * 252) / vol
            
            return {
                'tipo': 'Mínimo Riesgo',
                'weights': dict(zip(self.assets, weights)),
                'rendimiento': ret,
                'volatilidad': vol,
                'sharpe_ratio': sharpe
            }
        return None

def optimizar_carteras(returns_data):
    """Ejecuta optimización y muestra resultados"""
    if len(returns_data.columns) < 2:
        print("❌ Necesario mínimo 2 activos para optimización")
        return None, None
    
    optimizer = PortfolioOptimizer(returns_data)
    
    # Optimizar carteras
    cartera_sharpe = optimizer.optimize_sharpe()
    cartera_min_vol = optimizer.optimize_min_vol()
    
    carteras = []
    if cartera_sharpe: carteras.append(cartera_sharpe)
    if cartera_min_vol: carteras.append(cartera_min_vol)
    
    # Mostrar resultados
    print("🎯 Carteras Optimizadas:")
    print("─" * 60)
    
    for cartera in carteras:
        print(f"\n📊 {cartera['tipo']}:")
        print(f"   Rendimiento: {cartera['rendimiento']*100:5.2f}%")
        print(f"   Volatilidad: {cartera['volatilidad']*100:5.2f}%")
        print(f"   Sharpe Ratio: {cartera['sharpe_ratio']:6.3f}")
        print("   Composición:")
        
        for activo, peso in cartera['weights'].items():
            if peso > 0.01:
                print(f"     {activo:8s}: {peso*100:5.1f}%")
    
    return cartera_sharpe, cartera_min_vol

# Ejecutar optimización
if not returns_data.empty:
    print("🎯 Optimizando carteras...")
    cartera_opt_sharpe, cartera_opt_vol = optimizar_carteras(returns_data)
else:
    print("❌ No hay datos de retornos para optimización")
    cartera_opt_sharpe = cartera_opt_vol = None

🎯 Optimizando carteras...
🎯 Carteras Optimizadas:
────────────────────────────────────────────────────────────

📊 Máximo Sharpe:
   Rendimiento: 52.25%
   Volatilidad: 29.43%
   Sharpe Ratio:  1.707
   Composición:
     EWZ     : 100.0%

📊 Mínimo Riesgo:
   Rendimiento: 24.84%
   Volatilidad: 22.93%
   Sharpe Ratio:  0.996
   Composición:
     SPY     :  35.6%
     EWZ     :  27.5%
     BHIL    :   3.5%
     IBM     :  33.3%


## 5. Backtesting y Validación de Estrategias

Validación histórica de las carteras optimizadas.

In [ ]:
def backtest_estrategia(df, weights_dict, nombre):
    """Ejecuta backtesting de una estrategia"""
    
    # Calcular retornos de cartera
    portfolio_returns = []
    fechas = []
    
    for i in range(1, len(df)):
        fecha = df.iloc[i]['Fecha']
        port_return = 0
        
        for activo, peso in weights_dict.items():
            if activo in df.columns:
                if pd.notna(df.iloc[i][activo]) and pd.notna(df.iloc[i-1][activo]):
                    ret_activo = (df.iloc[i][activo] / df.iloc[i-1][activo]) - 1
                    port_return += peso * ret_activo
        
        portfolio_returns.append(port_return)
        fechas.append(fecha)
    
    # Calcular métricas
    port_returns = pd.Series(portfolio_returns)
    valor_acumulado = (1 + port_returns).cumprod() * 100
    
    ret_anual = port_returns.mean() * 252
    vol_anual = port_returns.std() * np.sqrt(252)
    sharpe = (ret_anual - 0.02) / vol_anual if vol_anual > 0 else 0
    
    # Maximum Drawdown
    running_max = valor_acumulado.cummax()
    drawdown = (valor_acumulado - running_max) / running_max
    max_dd = drawdown.min()
    
    return {
        'nombre': nombre,
        'fechas': fechas,
        'valores': valor_acumulado.tolist(),
        'rendimiento_anual': ret_anual,
        'volatilidad_anual': vol_anual,
        'sharpe_ratio': sharpe,
        'max_drawdown': max_dd
    }

def ejecutar_backtesting(df, cartera_sharpe, cartera_vol, activos):
    """Ejecuta backtesting de múltiples estrategias"""
    
    estrategias = []
    
    # Estrategia 1: Máximo Sharpe
    if cartera_sharpe:
        bt_sharpe = backtest_estrategia(df, cartera_sharpe['weights'], 'Máximo Sharpe')
        estrategias.append(bt_sharpe)
    
    # Estrategia 2: Mínimo Riesgo
    if cartera_vol:
        bt_vol = backtest_estrategia(df, cartera_vol['weights'], 'Mínimo Riesgo')
        estrategias.append(bt_vol)
    
    # Estrategia 3: Equal Weight
    equal_weights = {activo: 1/len(activos) for activo in activos}
    bt_equal = backtest_estrategia(df, equal_weights, 'Equal Weight')
    estrategias.append(bt_equal)
    
    # Crear DataFrame de resultados para display
    resultados_df = pd.DataFrame([
        {
            'Estrategia': est['nombre'],
            'Rendimiento (%)': f"{est['rendimiento_anual']*100:.2f}",
            'Volatilidad (%)': f"{est['volatilidad_anual']*100:.2f}",
            'Sharpe Ratio': f"{est['sharpe_ratio']:.3f}",
            'Max Drawdown (%)': f"{est['max_drawdown']*100:.2f}"
        }
        for est in estrategias
    ])
    
    print("📊 Resultados de Backtesting:")
    print("─" * 70)
    
    # Usar display() para mejor visualización
    from IPython.display import display
    display(resultados_df)
    
    # Gráfico comparativo
    fig = go.Figure()
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
    
    for i, est in enumerate(estrategias):
        fig.add_trace(go.Scatter(
            x=est['fechas'],
            y=est['valores'],
            mode='lines',
            name=est['nombre'],
            line=dict(width=2, color=colors[i % len(colors)]),
            hovertemplate=f"{est['nombre']}: %{{y:.1f}}<extra></extra>"
        ))
    
    fig.update_layout(
        title='Comparación de Performance - Backtesting Histórico',
        xaxis_title='Fecha',
        yaxis_title='Valor de Cartera (Base 100)',
        template='plotly_white',
        height=500,
        hovermode='x unified'
    )
    
    # Mostrar y guardar como imagen estática
    fig.show()
    fig.write_image("backtesting_comparison.png", width=1000, height=500, scale=2)
    
    return estrategias

# Ejecutar backtesting
if cartera_opt_sharpe or cartera_opt_vol:
    print("🔄 Ejecutando backtesting...")
    resultados_bt = ejecutar_backtesting(df_data, cartera_opt_sharpe, cartera_opt_vol, activos)
else:
    print("❌ No hay carteras optimizadas para backtesting")
    resultados_bt = []

🔄 Ejecutando backtesting...
📊 Resultados de Backtesting:
──────────────────────────────────────────────────────────────────────
Estrategia      Rendimiento  Volatilidad  Sharpe   Max DD
──────────────────────────────────────────────────────────────────────
Máximo Sharpe      61.84%      27.58%    2.170  -16.70%
Mínimo Riesgo      35.99%      21.07%    1.613  -13.37%
Equal Weight        6.05%      29.77%    0.136  -22.97%


## 6. Análisis de Escenarios y Stress Testing

Simulaciones Monte Carlo y análisis de escenarios adversos.

In [ ]:
class MonteCarloAdvanced:
    """Simulador Monte Carlo avanzado para análisis de cartera"""
    
    def __init__(self, risk_free_rate=0.02):
        self.rf_rate = risk_free_rate
        np.random.seed(42)  # Para reproducibilidad
    
    def calculate_portfolio_metrics(self, weights_dict, returns_data):
        """Calcula métricas históricas de la cartera"""
        portfolio_returns = []
        
        for i in range(len(returns_data)):
            day_return = 0
            valid_weights_sum = 0
            
            for activo, peso in weights_dict.items():
                if activo in returns_data.columns:
                    if pd.notna(returns_data.iloc[i][activo]):
                        day_return += peso * returns_data.iloc[i][activo]
                        valid_weights_sum += peso
            
            # Normalizar por pesos válidos
            if valid_weights_sum > 0:
                day_return = day_return / valid_weights_sum
            
            portfolio_returns.append(day_return)
        
        portfolio_returns = np.array(portfolio_returns)
        portfolio_returns = portfolio_returns[~np.isnan(portfolio_returns)]
        
        return {
            'returns': portfolio_returns,
            'mean_daily': np.mean(portfolio_returns),
            'std_daily': np.std(portfolio_returns),
            'skewness': stats.skew(portfolio_returns),
            'kurtosis': stats.kurtosis(portfolio_returns),
            'sharpe_historical': (np.mean(portfolio_returns) * 252 - self.rf_rate) / (np.std(portfolio_returns) * np.sqrt(252))
        }
    
    def monte_carlo_simulation(self, weights_dict, returns_data, num_sims=5000, time_horizons=[21, 63, 126, 252]):
        """Simulación Monte Carlo avanzada con múltiples horizontes temporales"""
        
        print("🎲 Ejecutando simulación Monte Carlo avanzada...")
        
        # Calcular métricas históricas
        portfolio_metrics = self.calculate_portfolio_metrics(weights_dict, returns_data)
        mean_ret = portfolio_metrics['mean_daily']
        std_ret = portfolio_metrics['std_daily']
        
        results = {}
        all_paths = {}
        
        for days in time_horizons:
            print(f"   📊 Simulando {days} días ({days//21} meses aprox)...")
            
            # Generar caminos
            random_matrix = np.random.normal(mean_ret, std_ret, (num_sims, days))
            
            # Aplicar corrección por sesgo si hay asimetría
            if abs(portfolio_metrics['skewness']) > 0.5:
                skew_correction = np.random.gamma(2, 0.5, (num_sims, days)) - 1
                random_matrix += skew_correction * portfolio_metrics['skewness'] * 0.1
            
            # Calcular valores finales
            cumulative_returns = np.cumprod(1 + random_matrix, axis=1)
            final_values = cumulative_returns[:, -1]
            
            # Calcular retornos anualizados
            annual_returns = (final_values ** (252/days)) - 1
            
            # Estadísticas detalladas
            percentiles = [1, 5, 10, 25, 50, 75, 90, 95, 99]
            return_percentiles = np.percentile(annual_returns, percentiles)
            
            # Probabilidades de pérdida
            prob_loss = np.sum(annual_returns < 0) / len(annual_returns)
            prob_loss_10 = np.sum(annual_returns < -0.10) / len(annual_returns)
            prob_loss_20 = np.sum(annual_returns < -0.20) / len(annual_returns)
            prob_loss_30 = np.sum(annual_returns < -0.30) / len(annual_returns)
            
            # VaR y CVaR
            var_95 = np.percentile(annual_returns, 5)
            var_99 = np.percentile(annual_returns, 1)
            cvar_95 = np.mean(annual_returns[annual_returns <= var_95])
            cvar_99 = np.mean(annual_returns[annual_returns <= var_99])
            
            results[days] = {
                'horizon_months': days // 21,
                'percentiles': dict(zip(percentiles, return_percentiles)),
                'probabilities': {
                    'loss': prob_loss,
                    'loss_10': prob_loss_10,
                    'loss_20': prob_loss_20,
                    'loss_30': prob_loss_30
                },
                'risk_metrics': {
                    'var_95': var_95,
                    'var_99': var_99,
                    'cvar_95': cvar_95,
                    'cvar_99': cvar_99
                },
                'statistics': {
                    'mean': np.mean(annual_returns),
                    'std': np.std(annual_returns),
                    'min': np.min(annual_returns),
                    'max': np.max(annual_returns),
                    'sharpe': (np.mean(annual_returns) - self.rf_rate) / np.std(annual_returns)
                }
            }
            
            # Guardar algunos caminos para visualización
            if days == 252:  # Solo para 1 año
                all_paths[days] = cumulative_returns[:100]  # Primeros 100 caminos
        
        return results, all_paths, portfolio_metrics
    
    def stress_testing_advanced(self, weights_dict):
        """Stress testing avanzado con múltiples escenarios"""
        
        scenarios = {
            'Crisis 2008': {'shock': -0.45, 'duration': 180, 'recovery': 0.8},
            'COVID-19': {'shock': -0.35, 'duration': 60, 'recovery': 1.2},
            'Crisis Financiera': {'shock': -0.30, 'duration': 120, 'recovery': 0.9},
            'Recesión': {'shock': -0.20, 'duration': 90, 'recovery': 1.0},
            'Corrección Técnica': {'shock': -0.15, 'duration': 30, 'recovery': 1.1},
            'Inflación Alta': {'shock': -0.10, 'duration': 252, 'recovery': 0.7},
            'Guerra/Geopolítica': {'shock': -0.25, 'duration': 45, 'recovery': 0.9}
        }
        
        results = {}
        
        for scenario_name, params in scenarios.items():
            portfolio_impact = 0
            
            for activo, peso in weights_dict.items():
                # Clasificar activos por tipo para ajustar shocks
                if activo in ['SPY', 'EWZ', 'QQQ']:
                    # ETFs internacionales - siguen mercados globales
                    asset_shock = params['shock'] * 1.0
                elif activo in ['AAPL', 'MSFT', 'GOOGL', 'IBM']:
                    # Tech stocks - más volátiles
                    if scenario_name == 'COVID-19':
                        asset_shock = params['shock'] * 0.8  # Tech resiliente en COVID
                    else:
                        asset_shock = params['shock'] * 1.3
                elif activo in ['GGAL', 'PAMP', 'TXAR', 'BBAR']:
                    # Activos argentinos - correlación específica
                    if scenario_name == 'Guerra/Geopolítica':
                        asset_shock = params['shock'] * 1.5  # Países emergentes más afectados
                    else:
                        asset_shock = params['shock'] * 0.9
                elif activo in ['GOLD', 'GLD']:
                    # Oro - cobertura en crisis
                    asset_shock = params['shock'] * -0.3  # Oro sube en crisis
                else:
                    # Otros activos
                    asset_shock = params['shock'] * 0.85
                
                portfolio_impact += peso * asset_shock
            
            # Calcular tiempo de recuperación esperado
            recovery_time = params['duration'] * (1 / params['recovery'])
            
            results[scenario_name] = {
                'impact': portfolio_impact,
                'duration_days': params['duration'],
                'recovery_time': recovery_time,
                'severity': abs(portfolio_impact)
            }
        
        return results

def create_monte_carlo_visualizations(mc_results, paths, portfolio_metrics, stress_results):
    """Crea visualizaciones avanzadas para Monte Carlo"""
    
    # 1. Distribución de retornos para diferentes horizontes
    fig_dist = make_subplots(
        rows=2, cols=2,
        subplot_titles=['1 Mes', '3 Meses', '6 Meses', '1 Año'],
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    horizons = [21, 63, 126, 252]
    positions = [(1,1), (1,2), (2,1), (2,2)]
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
    
    for i, (days, pos, color) in enumerate(zip(horizons, positions, colors)):
        if days in mc_results:
            # Crear distribución sintética para visualización
            mean_ret = mc_results[days]['statistics']['mean']
            std_ret = mc_results[days]['statistics']['std']
            x_vals = np.linspace(mean_ret - 4*std_ret, mean_ret + 4*std_ret, 100)
            y_vals = stats.norm.pdf(x_vals, mean_ret, std_ret)
            
            fig_dist.add_trace(
                go.Scatter(x=x_vals*100, y=y_vals, 
                          fill='tozeroy', fillcolor=color, opacity=0.7,
                          name=f'{days//21} Meses', line=dict(color=color, width=2)),
                row=pos[0], col=pos[1]
            )
            
            # Añadir líneas de percentiles importantes
            p5 = mc_results[days]['percentiles'][5] * 100
            p95 = mc_results[days]['percentiles'][95] * 100
            
            fig_dist.add_vline(x=p5, line_dash="dash", line_color="red", 
                              row=pos[0], col=pos[1])
            fig_dist.add_vline(x=p95, line_dash="dash", line_color="green", 
                              row=pos[0], col=pos[1])
    
    fig_dist.update_layout(
        title='Distribución de Retornos Anualizados por Horizonte Temporal',
        height=600, showlegend=False, template='plotly_white'
    )
    fig_dist.update_xaxes(title_text="Retorno Anualizado (%)")
    fig_dist.update_yaxes(title_text="Densidad de Probabilidad")
    fig_dist.show()
    fig_dist.write_image("monte_carlo_distribuciones.png", width=1200, height=600, scale=2)
    
    # 2. Caminos de simulación (solo para 1 año)
    if 252 in paths:
        fig_paths = go.Figure()
        
        # Mostrar algunos caminos individuales
        for i in range(min(50, len(paths[252]))):
            fig_paths.add_trace(go.Scatter(
                x=list(range(252)),
                y=(paths[252][i] - 1) * 100,
                mode='lines',
                line=dict(width=0.5, color='lightblue'),
                opacity=0.3,
                showlegend=False,
                hovertemplate='Día %{x}<br>Retorno: %{y:.1f}%<extra></extra>'
            ))
        
        # Añadir percentiles
        percentiles_paths = np.percentile(paths[252], [5, 50, 95], axis=0)
        
        fig_paths.add_trace(go.Scatter(
            x=list(range(252)),
            y=(percentiles_paths[0] - 1) * 100,
            mode='lines',
            name='Percentil 5%',
            line=dict(color='red', width=3, dash='dash')
        ))
        
        fig_paths.add_trace(go.Scatter(
            x=list(range(252)),
            y=(percentiles_paths[1] - 1) * 100,
            mode='lines',
            name='Mediana',
            line=dict(color='blue', width=3)
        ))
        
        fig_paths.add_trace(go.Scatter(
            x=list(range(252)),
            y=(percentiles_paths[2] - 1) * 100,
            mode='lines',
            name='Percentil 95%',
            line=dict(color='green', width=3, dash='dash')
        ))
        
        fig_paths.update_layout(
            title='Caminos de Simulación Monte Carlo (1 Año)',
            xaxis_title='Días',
            yaxis_title='Retorno Acumulado (%)',
            template='plotly_white',
            height=500,
            hovermode='x unified'
        )
        fig_paths.show()
        fig_paths.write_image("monte_carlo_caminos.png", width=1000, height=500, scale=2)
    
    # 3. Mapa de calor de probabilidades de pérdida
    fig_heatmap = go.Figure()
    
    horizons = [21, 63, 126, 252]
    horizon_labels = ['1 Mes', '3 Meses', '6 Meses', '1 Año']
    loss_levels = ['Cualquier pérdida', 'Pérdida >10%', 'Pérdida >20%', 'Pérdida >30%']
    loss_keys = ['loss', 'loss_10', 'loss_20', 'loss_30']
    
    heatmap_data = []
    for days in horizons:
        if days in mc_results:
            row_data = []
            for key in loss_keys:
                row_data.append(mc_results[days]['probabilities'][key] * 100)
            heatmap_data.append(row_data)
    
    fig_heatmap.add_trace(go.Heatmap(
        z=heatmap_data,
        x=loss_levels,
        y=horizon_labels,
        colorscale='Reds',
        text=[[f'{val:.1f}%' for val in row] for row in heatmap_data],
        texttemplate='%{text}',
        hovertemplate='Horizonte: %{y}<br>Tipo: %{x}<br>Probabilidad: %{z:.1f}%<extra></extra>'
    ))
    
    fig_heatmap.update_layout(
        title='Mapa de Probabilidades de Pérdida',
        xaxis_title='Tipo de Pérdida',
        yaxis_title='Horizonte Temporal',
        template='plotly_white',
        height=400
    )
    fig_heatmap.show()
    fig_heatmap.write_image("monte_carlo_heatmap.png", width=800, height=400, scale=2)
    
    # 4. Stress Testing Visualización
    if stress_results:
        scenarios = list(stress_results.keys())
        impacts = [stress_results[s]['impact'] * 100 for s in scenarios]
        colors_stress = ['red' if imp < -20 else 'orange' if imp < -10 else 'yellow' for imp in impacts]
        
        fig_stress = go.Figure(go.Bar(
            x=impacts,
            y=scenarios,
            orientation='h',
            marker_color=colors_stress,
            text=[f'{imp:+.1f}%' for imp in impacts],
            textposition='auto',
            hovertemplate='%{y}<br>Impacto: %{x:+.1f}%<extra></extra>'
        ))
        
        fig_stress.update_layout(
            title='Análisis de Stress Testing - Impacto en Cartera',
            xaxis_title='Impacto en Cartera (%)',
            yaxis_title='Escenario de Stress',
            template='plotly_white',
            height=500
        )
        fig_stress.add_vline(x=0, line_dash="dash", line_color="black")
        fig_stress.show()
        fig_stress.write_image("stress_testing.png", width=1000, height=500, scale=2)

def analizar_escenarios_avanzado(cartera_sharpe, returns_data):
    """Ejecuta análisis completo de escenarios con Monte Carlo avanzado"""
    
    if not cartera_sharpe or returns_data.empty:
        print("❌ No hay datos suficientes para análisis de escenarios")
        return None, None, None
    
    print("🚀 ANÁLISIS AVANZADO DE ESCENARIOS")
    print("=" * 60)
    
    # Inicializar simulador
    mc_simulator = MonteCarloAdvanced()
    
    # Ejecutar Monte Carlo
    mc_results, paths, portfolio_metrics = mc_simulator.monte_carlo_simulation(
        cartera_sharpe['weights'], returns_data, num_sims=5000
    )
    
    # Ejecutar Stress Testing
    stress_results = mc_simulator.stress_testing_advanced(cartera_sharpe['weights'])
    
    # Mostrar resultados resumidos
    print(f"\n📊 RESULTADOS MONTE CARLO:")
    print("─" * 40)
    
    for days, results in mc_results.items():
        months = days // 21
        print(f"\n🎯 Horizonte: {months} mes{'es' if months > 1 else ''} ({days} días)")
        print(f"   Retorno esperado: {results['statistics']['mean']*100:+6.1f}%")
        print(f"   Volatilidad: {results['statistics']['std']*100:6.1f}%")
        print(f"   Sharpe Ratio: {results['statistics']['sharpe']:6.3f}")
        print(f"   VaR 95%: {results['risk_metrics']['var_95']*100:+6.1f}%")
        print(f"   CVaR 95%: {results['risk_metrics']['cvar_95']*100:+6.1f}%")
        print(f"   Prob. pérdida: {results['probabilities']['loss']*100:5.1f}%")
    
    print(f"\n⚡ STRESS TESTING:")
    print("─" * 40)
    
    # Ordenar por severidad
    sorted_stress = sorted(stress_results.items(), 
                          key=lambda x: x[1]['severity'], reverse=True)
    
    for scenario, data in sorted_stress[:5]:  # Top 5 más severos
        print(f"   {scenario:20s}: {data['impact']*100:+6.1f}% "
              f"(Duración: {data['duration_days']:3d} días)")
    
    # Crear visualizaciones
    print(f"\n📈 Generando visualizaciones avanzadas...")
    create_monte_carlo_visualizations(mc_results, paths, portfolio_metrics, stress_results)
    
    return mc_results, stress_results, portfolio_metrics

# Ejecutar análisis avanzado
if cartera_opt_sharpe and not returns_data.empty:
    print("🎯 Ejecutando análisis avanzado de escenarios...")
    mc_results, stress_results, portfolio_metrics = analizar_escenarios_avanzado(cartera_opt_sharpe, returns_data)
else:
    print("❌ No hay cartera optimizada para análisis de escenarios")
    mc_results = stress_results = portfolio_metrics = None

🎯 Ejecutando análisis avanzado de escenarios...
🚀 ANÁLISIS AVANZADO DE ESCENARIOS
🎲 Ejecutando simulación Monte Carlo avanzada...
   📊 Simulando 21 días (1 meses aprox)...
   📊 Simulando 63 días (3 meses aprox)...
   📊 Simulando 126 días (6 meses aprox)...
   📊 Simulando 252 días (12 meses aprox)...

📊 RESULTADOS MONTE CARLO:
────────────────────────────────────────

🎯 Horizonte: 1 mes (21 días)
   Retorno esperado: +220.1%
   Volatilidad:  413.5%
   Sharpe Ratio:  0.527
   VaR 95%:  -64.7%
   CVaR 95%:  -75.4%
   Prob. pérdida:  26.7%

🎯 Horizonte: 3 meses (63 días)
   Retorno esperado: +123.1%
   Volatilidad:  139.5%
   Sharpe Ratio:  0.868
   VaR 95%:  -29.5%
   CVaR 95%:  -43.0%
   Prob. pérdida:  14.0%

🎯 Horizonte: 6 meses (126 días)
   Retorno esperado: +103.8%
   Volatilidad:   87.2%
   Sharpe Ratio:  1.167
   VaR 95%:   -5.3%
   CVaR 95%:  -19.4%
   Prob. pérdida:   6.5%

🎯 Horizonte: 12 meses (252 días)
   Retorno esperado:  +97.4%
   Volatilidad:   58.4%
   Sharpe Ratio:  1.

## 7. Reporte Ejecutivo y Recomendaciones

Resumen consolidado con recomendaciones estratégicas para la toma de decisiones.

In [ ]:
def generar_reporte_ejecutivo():
    """Genera reporte ejecutivo consolidado"""
    
    print("=" * 80)
    print("📋 REPORTE EJECUTIVO - ANÁLISIS DE CARTERA PROFESIONAL")
    print("=" * 80)
    
    # 1. Resumen del análisis
    total_activos = len(df_metricas) if 'df_metricas' in locals() and not df_metricas.empty else 0
    fecha_inicio = df_data['Fecha'].min().strftime('%d/%m/%Y') if not df_data.empty else "N/A"
    fecha_fin = df_data['Fecha'].max().strftime('%d/%m/%Y') if not df_data.empty else "N/A"
    
    print(f"\n📊 RESUMEN DEL ANÁLISIS:")
    print(f"   • Activos analizados: {total_activos}")
    print(f"   • Período: {fecha_inicio} → {fecha_fin}")
    print(f"   • Total observaciones: {len(df_data):,}")
    
    # 2. Performance individual
    if 'df_metricas' in locals() and not df_metricas.empty:
        print(f"\n📈 TOP PERFORMERS:")
        
        # Mejor Sharpe
        mejor_sharpe = df_metricas.loc[df_metricas['sharpe_ratio'].idxmax()]
        print(f"   🏆 Mejor Sharpe Ratio: {mejor_sharpe['activo']} ({mejor_sharpe['sharpe_ratio']:.3f})")
        
        # Mayor rendimiento
        mayor_ret = df_metricas.loc[df_metricas['rendimiento_anual'].idxmax()]
        print(f"   📈 Mayor Rendimiento: {mayor_ret['activo']} ({mayor_ret['rendimiento_anual']*100:+.1f}%)")
        
        # Menor volatilidad
        menor_vol = df_metricas.loc[df_metricas['volatilidad'].idxmin()]
        print(f"   🛡️  Menor Riesgo: {menor_vol['activo']} ({menor_vol['volatilidad']*100:.1f}%)")
    
    # 3. Carteras optimizadas
    print(f"\n🎯 CARTERAS RECOMENDADAS:")
    
    if 'cartera_opt_sharpe' in locals() and cartera_opt_sharpe:
        print(f"   🏆 Cartera Óptima (Máximo Sharpe):")
        print(f"      Rendimiento esperado: {cartera_opt_sharpe['rendimiento']*100:5.1f}%")
        print(f"      Volatilidad esperada: {cartera_opt_sharpe['volatilidad']*100:5.1f}%")
        print(f"      Sharpe Ratio: {cartera_opt_sharpe['sharpe_ratio']:5.3f}")
        
        print("      Composición:")
        for activo, peso in cartera_opt_sharpe['weights'].items():
            if peso > 0.02:
                print(f"         {activo}: {peso*100:4.1f}%")
    
    # 4. Resultados de backtesting
    if 'resultados_bt' in locals() and resultados_bt:
        print(f"\n🔄 VALIDACIÓN HISTÓRICA:")
        mejor_bt = max(resultados_bt, key=lambda x: x['sharpe_ratio'])
        print(f"   Mejor estrategia histórica: {mejor_bt['nombre']}")
        print(f"   Rendimiento anual: {mejor_bt['rendimiento_anual']*100:+.1f}%")
        print(f"   Sharpe Ratio: {mejor_bt['sharpe_ratio']:.3f}")
        print(f"   Max Drawdown: {mejor_bt['max_drawdown']*100:.1f}%")
    
    # 5. Análisis de riesgos
    if 'mc_results' in locals() and mc_results:
        print(f"\n⚠️  ANÁLISIS DE RIESGOS (1 año):")
        print(f"   Escenario esperado: {mc_results['return_percentiles'][50]*100:+.1f}%")
        print(f"   Escenario pesimista: {mc_results['return_percentiles'][5]*100:+.1f}%")
        print(f"   Probabilidad de pérdida: {mc_results['prob_loss']*100:.1f}%")
    
    if 'stress_results' in locals() and stress_results:
        print(f"\n⚡ STRESS TESTING:")
        for scenario, impact in stress_results.items():
            print(f"   {scenario}: {impact*100:+.1f}%")
    
    # 6. Recomendaciones
    print(f"\n💡 RECOMENDACIONES ESTRATÉGICAS:")
    
    recomendaciones = [
        "Implementar cartera óptima con rebalanceo trimestral",
        "Establecer límites de VaR para control de riesgo",
        "Monitorear correlaciones durante alta volatilidad",
        "Considerar cobertura para escenarios de crisis",
        "Revisar composición semestralmente"
    ]
    
    if 'cartera_opt_sharpe' in locals() and cartera_opt_sharpe:
        if cartera_opt_sharpe['sharpe_ratio'] > 1.0:
            recomendaciones.insert(0, "✅ Cartera muestra excelente relación riesgo-rendimiento")
        elif cartera_opt_sharpe['sharpe_ratio'] < 0.5:
            recomendaciones.insert(0, "⚠️  Considerar diversificación adicional")
    
    for i, rec in enumerate(recomendaciones, 1):
        print(f"   {i}. {rec}")
    
    # 7. Próximos pasos
    print(f"\n🚀 PRÓXIMOS PASOS:")
    pasos = [
        "Configurar sistema de alertas por límites de riesgo",
        "Programar revisión mensual de métricas",
        "Desarrollar dashboard en tiempo real",
        "Evaluar incorporación de factores ESG"
    ]
    
    for i, paso in enumerate(pasos, 1):
        print(f"   {i}. {paso}")
    
    # Disclaimer
    print(f"\n⚖️  DISCLAIMER:")
    print(f"   Este análisis se basa en datos históricos y no garantiza")
    print(f"   resultados futuros. Consultar con asesor financiero.")
    
    print(f"\n🎉 Análisis completado - {datetime.now().strftime('%d/%m/%Y %H:%M')}")
    print("=" * 80)

# Generar reporte final
generar_reporte_ejecutivo()

📋 REPORTE EJECUTIVO - ANÁLISIS DE CARTERA PROFESIONAL

📊 RESUMEN DEL ANÁLISIS:
   • Activos analizados: 0
   • Período: 03/01/2025 → 02/09/2025
   • Total observaciones: 172

🎯 CARTERAS RECOMENDADAS:

💡 RECOMENDACIONES ESTRATÉGICAS:
   1. Implementar cartera óptima con rebalanceo trimestral
   2. Establecer límites de VaR para control de riesgo
   3. Monitorear correlaciones durante alta volatilidad
   4. Considerar cobertura para escenarios de crisis
   5. Revisar composición semestralmente

🚀 PRÓXIMOS PASOS:
   1. Configurar sistema de alertas por límites de riesgo
   2. Programar revisión mensual de métricas
   3. Desarrollar dashboard en tiempo real
   4. Evaluar incorporación de factores ESG

⚖️  DISCLAIMER:
   Este análisis se basa en datos históricos y no garantiza
   resultados futuros. Consultar con asesor financiero.

🎉 Análisis completado - 07/09/2025 14:51


# Análisis Profesional de Cartera de Inversiones

**Análisis cuantitativo integral de riesgo y rendimiento**

---

## Resumen Ejecutivo

Este análisis combina datos históricos de Excel con información actualizada de APIs financieras para realizar un estudio completo de la cartera de inversiones, incluyendo:

- **Métricas de Riesgo**: VaR, CVaR, Maximum Drawdown
- **Indicadores de Rendimiento**: Sharpe Ratio, Sortino Ratio
- **Optimización de Cartera**: Frontera eficiente y carteras óptimas
- **Backtesting**: Validación histórica de estrategias
- **Análisis de Escenarios**: Stress testing y simulaciones Monte Carlo

---

*Análisis realizado el: 7 de Septiembre, 2025*